Modelos baseados em ávore são modelos não lineares, não paramétricos bem conhecidos dentre os modelos de aprendizado surpervisionado. Além disso, eles facilitam a interpretação, tem uma predição rápida (ao contrário do kNN). Por fim, eles resolvem bem tanto problemas de classificação quanto de regressão.

![title](../data/imgs/boosting0.png)

## Revisando conceitos

Árvores de dcisão são um tipo de algoritmo de aprendizado surpervisionado que funciona tanto para varáveis categóricas quanto contítunas. Nessa tecnica, nós dividimos a população (ou a amostra) em sub grupos homongêneos baseado no `split` que nos dê o maior ganho de informação com base em cada uma das variáveis de entrada.

![title](../data/imgs/boosting1.png)

Fun fact: exemplo de uma árvore de decisão
http://pt.akinator.com/

![title](../data/imgs/boosting2.png)

Árvores de decisao podem ser usadas tanto para classificação quanto para regressão. No caso da regressão, o valor obtido por uma folha no dado de treino é basicamente a média das observações que cairam naquela região. Portanto, se um dado não visto cair nessa região, faremos sua previsão com base nos valores treinados.

No caso de classificação, o valor da folha é a moda das categorias.

Nos dois casos, o processo de divisão (splitting) da ávore ocorre na ávore toda até algum critério de parada (por exemplo, número máximo de splits ou profundidade da árvore) sejam atingidos. Contudo, caso não haja `prunning`, ou limitar alguns parâmetros, é normal que ocorra `overfitting` dos dados.

## Relembrando hiperparâmetros

![title](../data/imgs/boosting3.png)

### O dataset
O dataset foi originalmente tirado do National Institute of Diabetes and Digestive and Kidney Diseases. Seu objetivo é diagnosticar se um paciente têm ou não diabetes, baseado em certas medidas de diagnóstico incluídas no dataset. Em particular, todos os paciantes aqui são mulheres de pelo menos 21 anos de idade.

Vamos prever se conseguimos descobrir se os pacientes têm diabetes?

In [3]:
import pandas as pd
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv(url, names=names)
dataframe.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [18]:
#Import Library
#Import other necessary libraries like pandas, numpy...
from sklearn import tree
from sklearn.model_selection import train_test_split
# Create tree object 
model = tree.DecisionTreeClassifier(criterion='gini') # for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini  
# model = tree.DecisionTreeRegressor() for regression
# Train the model using the training sets and check score
array = dataframe.values
X = array[:,0:8]
y = array[:,8]
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=7, stratify=y, test_size=0.2)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.70779220779220775

O problema das Decision Trees é que elas tendem ao `overfit` e isso ocorre ainda pode ocorrer com prunning.

![title](../data/imgs/boosting4.png)

A ideia é que ao invés de usarmos uma árvore única, combinarmos árvores mais "fracas"/menores para construir noss modelo?

![title](../data/imgs/boosting5.png)


The literary meaning of word ‘ensemble’ is group. Ensemble methods involve group of predictive models to achieve a better accuracy and model stability. Ensemble methods are known to impart supreme boost to tree based models.

Like every other model, a tree based model also suffers from the plague of bias and variance. Bias means, ‘how much on an average are the predicted values different from the actual value.’ Variance means, ‘how different will the predictions of the model be at the same point if different samples are taken from the same population’.

You build a small tree and you will get a model with low variance and high bias. How do you manage to balance the trade off between bias and variance ?

Normally, as you increase the complexity of your model, you will see a reduction in prediction error due to lower bias in the model. As you continue to make your model more complex, you end up over-fitting your model and your model will start suffering from high variance.

A champion model should maintain a balance between these two types of errors. This is known as the trade-off management of bias-variance errors. Ensemble learning is one way to execute this trade off analysis.

## Ensemble

É aí que entra o Ensemble. Métodos de ensemble são métodos que envolvem agrupar modelos preditivos para atingir uma melhor estabilidade ao combinar classificadores mais "fracos". 
Fun fact: O time ganhador do [Netflix prize](https://www.youtube.com/watch?v=ImpV70uLxyw&t=192s) aplicou cerca de 800 algoritmos combinados !

Assim como vimos em `kNN`, modelos de árvore também podem sofrer de excesso de bias (muito genérico/pouco treinado) e variance (muito complexo, tende ao overfit). Técnicas de Ensemble foram desenvolvidas para combater esse trade-off de bias vs variance.

![title](../data/imgs/boosting6.png)

Bagging is a technique used to reduce the variance of our predictions by combining the result of multiple classifiers modeled on different sub-samples of the same data set. The following figure will make it clearer:

Como é possível ver na imagem acima, Bagging é uma tecnica usada para reduzir a variância das nossas predições em um dataset ao combinar o resultados de multiplos classificadores baseados em diversas amostras (com repetição) do mesmo dataset.

![title](../data/imgs/boosting7.png)

Create Multiple DataSets:
Sampling is done with replacement on the original data and new datasets are formed.
The new data sets can have a fraction of the columns as well as rows, which are generally hyper-parameters in a bagging model
Taking row and column fractions less than 1 helps in making robust models, less prone to overfitting
Build Multiple Classifiers:
Classifiers are built on each data set.
Generally the same classifier is modeled on each data set and predictions are made.
Combine Classifiers:
The predictions of all the classifiers are combined using a mean, median or mode value depending on the problem at hand.
The combined values are generally more robust than a single model.

- Criar os multiplos datasets:

  - Sampling with replacement do dataset original, formando datasets novos
  - O número de features por dataset e o número de dados por dataset viram **hiperparâmetros**
- Treine diversos modelos:

  - Um para cada dataset (Pela imagem acima, cada Model é treinado para um draw)
  - Geralmente é usado o mesmo tipo de classificador
  
- Combine os classificadores:
  - As previsões/combinação de todos os classificadores é feita por meio da média, mediana ou moda, dependendo do problema trabalhado

  

In [22]:
from sklearn.ensemble import BaggingClassifier
cart = tree.DecisionTreeClassifier(criterion='gini')
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=7)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.77922077922077926

![title](../data/imgs/boosting8.png)

## Mas e as Florestas Aleatórias

Florestas Aleatórias (ou Random Forests) são uma extensão de Bagging aplicado a Decision Trees.

Nela, os samples são treinados no dataset, mas além disso, as árvores são construídas de forma que os classificadores tenham sua correlação reduzida entre eles. Em outras palavras, ao invés de escolher a melhor divisão considerando todos os atributos disponíveis. Nós escolhemos o melhor com base em um subset de features (todos os subsets têm o mesmo tamanho), selecionados de forma aleatória.

![title](../data/imgs/boosting9.png)

In [24]:
from sklearn.ensemble import RandomForestClassifier
seed = 7
num_trees = 100
max_features = 3
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.75974025974025972

## Vantagens e Desvantagens da Random Forest

- Consegue lidar com datasets grandes e com muita dimensionalidade
- Os modelos conseguem definir a importância de cada variável !
- É capaz de lidar com dados faltantes


- Limitações para problemas de regressão: não consegue extrapolar dados para além do que foi treinado

![title](../data/imgs/boosting10.png)

## E para lidar com os casos de alto bias?

### Boosting !

Pela definição, o termo `Boosting` se refere à família de algoritmos que convertem um conjunto de `weak` learners e um único `strong` learner.


Pegando o exemplo anterior, suponha que queremos trabalhar com o problema de SPAM. Isto é queremos classificar se um email é SPAM ou não.

Sendo assim, como que classificaríamos emails como sendo de SPAM?

- Email que têm apenas uma imagem (de promoção) é SPAM
- Email que tem apenas links, é SPAM
- Email cujo corpo consiste de uma sentença como "Você ganhou na mega-sena", é SPAM
- Email do somostera, não é spam
- Email de uma fonte conhecida, não é spam

Definimos uma série de regras para classificar um email como `spam` ou não. Mas você acha que essas regras individualmente são fortes o bastante para classificar um email? Não.

Para combinar esses `weak learner` em um `strong learner` vamos utilizar métodos como:
- Usar uma média ponderada
- Considerar com base nas features, o `majoroty vote`
Por exemplo, se um dado não visto cair em 3 dos 5 critérios acima, ele será considerado como SPAM.

### Mas e como isso funciona?

Beleza, boosting combina uma série de weak learners de forma a formar um modelo só. Mas como que boosting identifica essas "regras fracas"?

No caso de árvores, técnicas de boosting se baseiam no que é chamado de "Stump tree", ou seja, "árvores com apenas uma divisão". A cada iteração que o algoritmo é aplicado, cria-se uma nova divisão com base nos erros do passado (o modelo "presta mais atenção" nos erros que tivemos). Sendo assim, após várias iterações, essas "weak rules" são combinadas, formando um modelo final.

Sendo assim, pode-se dizer que Boosting tem uma atenção maior em modelos classificados incorretamente.


![title](../data/imgs/boosting11.png)

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

seed = 7
num_trees = 100
max_features = 3
model = GradientBoostingClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.75324675324675328

É válido saber que os modelos de Boosting têm uma série de hiperparâmetros que não vimos aqui. Na segunda metade da aula vamos explorar outra biblioteca, que implementa o Gradient Boosting de uma forma mais eficiente que o Sklearn e que têm sido bastante famosa para ganhar várias competições de Data Science: o XgBoost